In [4]:
# imports
import requests
import pandas as pd
import os

In [2]:
# API Keys
YELP_API = "XZ-UMLXPORVPWbq6-U9Oo9g0d_2ShF4aYybxDaU9N8q8y9yNSILkSvW7pwfnIndjmh5qJV3xWNa0dI7pd_KigdckYq18c-OBveoSb7cygJf_4nwISszoadXzWZGnZ3Yx"
FOURSQUARE_API_KEY = "fsq3bxBWPRpRzw4UeoHiJZeY+OAzqHVhXy23pls8ppHrh60="

if not YELP_API or not FOURSQUARE_API_KEY:
    raise ValueError("API keys not found. Ensure YELP_API and FOURSQUARE_API_KEY are set as environment variables.")

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [16]:
# Load bike stations DataFrame
df_bike_stations = pd.read_csv("tokyo_bike_stations.csv")

# Convert bike stations to list of dictionaries (all stations, no limit)
station_locations = df_bike_stations.to_dict(orient="records")

In [18]:
# Foursquare API Function
def get_foursquare_results(latitude, longitude, category="bar"):
    url = "https://api.foursquare.com/v3/places/search"
    headers = {"Authorization": FOURSQUARE_API_KEY}
    params = {
        "query": category,
        "ll": f"{latitude},{longitude}",
        "radius": 1000,
        "limit": 50
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json().get("results", []) if response.status_code == 200 else []

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [19]:
# Collect data
foursquare_data = []

for station in station_locations:
    lat, lon = station["Latitude"], station["Longitude"]

    # Foursquare API results
    foursquare_results = get_foursquare_results(lat, lon)
    for place in foursquare_results:
        foursquare_data.append({
            "Bike Station": station["Name"],
            "Name": place["name"],
            "Latitude": place["geocodes"]["main"]["latitude"],
            "Longitude": place["geocodes"]["main"]["longitude"],
            "Category": place["categories"][0]["name"] if place.get("categories") else "N/A",
            "Address": place["location"].get("formatted_address", "N/A")
        })

Put your parsed results into a DataFrame

In [27]:
# Convert to Dataframe
df_foursquare = pd.DataFrame(foursquare_data)

# Drop Duplicates
df_foursquare = df_foursquare.drop_duplicates(subset=["Name", "Latitude", "Longitude"])

# Save to CSV
df_foursquare.to_csv("foursquare_results_final.csv", index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [24]:
# Yelp API Function
def get_yelp_results(latitude, longitude, term="bars"):
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": f"Bearer {YELP_API}"}
    params = {
        "term": term,  # Can be 'restaurants', 'bars', 'coffee'
        "latitude": latitude,
        "longitude": longitude,
        "radius": 1000,  # Search within a 1 km radius
        "limit": 50  # Max limit per request
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json().get("businesses", []) if response.status_code == 200 else []

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [25]:
# Collect Data
yelp_data = []

for station in station_locations:
    lat, lon = station["Latitude"], station["Longitude"]
    
    # Yelp API results
    yelp_results = get_yelp_results(lat, lon)
    for business in yelp_results:
        yelp_data.append({
            "Bike Station": station["Name"],
            "Name": business["name"],
            "Latitude": business["coordinates"]["latitude"],
            "Longitude": business["coordinates"]["longitude"],
            "Rating": business.get("rating", "N/A"),
            "Review Count": business.get("review_count", 0),
            "Address": ", ".join(business["location"]["display_address"])
        })

Put your parsed results into a DataFrame

In [28]:
# Convert to Dataframe
df_yelp = pd.DataFrame(yelp_data)

# Remove duplicates
df_yelp = df_yelp.drop_duplicates(subset=["Name", "Latitude", "Longitude"])

# Save to CSV
df_yelp.to_csv("yelp_results.csv", index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

```
Comparing the two results, Yelp provided the more complete information as it included all the data the Foursquare api did with rating and review count. The only difference is that the Foursquare API did provide the category specifying bar which Yelp did not provide
```

Get the top 10 restaurants according to their rating

In [30]:
# using the yelp results, This rating will not only go off of review but review count as well to add validity
df_yelp["Rating"] = pd.to_numeric(df_yelp["Rating"], errors="coerce")
df_yelp["Review Count"] = pd.to_numeric(df_yelp["Review Count"], errors="coerce")

In [31]:
# Remove rows with missing values in Rating or Review Count
df_yelp = df_yelp.dropna(subset=["Rating", "Review Count"])

In [32]:
# Sort by Rating (highest first) and Review Count (highest first)
df_top_bars = df_yelp.sort_values(by=["Rating", "Review Count"], ascending=[False, False]).head(10)

In [33]:
df_top_bars

,Bike Station,Name,Latitude,Longitude,Rating,Review Count,Address
8832,ロアール四谷,Shinjuku Cactus,35.691467,139.712848,5.0,49,"新宿1-30-12, ニューホワイトビルB1, Shinjuku, 東京都 〒160-002..."
23964,ル・サイク三田店,Bar Starman,35.662978,139.734341,5.0,15,"六本木3-13-3, プチビル3F, Minato, 東京都 〒106-0032, Japan"
17721,パラッツォ9ルーリヨ,BAR ASYL,35.693950,139.704856,5.0,11,"歌舞伎町1-1-8, Shinjuku, 東京都 〒160-0021, Japan"
15988,KM新宿ビル(西武新宿駅近辺),SHISHA TOKYO,35.699641,139.697791,5.0,10,"百人町1- 23-22, 2F, Shinjuku, 東京都 〒169-0073, Japan"
39260,京王プラザホテル,Edy's Bar,35.690058,139.699996,5.0,10,"歌舞伎町2-32-4, 柏崎造園ビル 1F, Shinjuku, 東京都 〒160-0021..."
10944,いちご西参道ビル,BAR Cocktail Book Shinjuku,35.696067,139.698053,5.0,9,"西新宿7-5-5, プラザ西新宿 2F, Shinjuku, 東京都 〒160-0023, ..."
14330,東洋カーマックス新宿御苑前,Bar Cinema Club,35.694136,139.704546,5.0,8,"歌舞伎町1-1-8, 花園3番街, Shinjuku, 東京都 〒160-0021, Japan"
2061,コインパーク平河町2丁目第2,Bar & Cafe Code Name Mixology Akasaka,35.673779,139.737427,5.0,7,"赤坂3-14-3, 渡林赤坂ビル 2F, Minato, 東京都 〒107-0052, Japan"
15975,KM新宿ビル(西武新宿駅近辺),Hidachisō Senya,35.699631,139.699907,5.0,7,"百人町1-9-14, Shinjuku, 東京都 〒169-0073, Japan"
169,アイパーク駐車場 世田谷代田南口第1,Beer Bar Ushitora Ichigo,35.662666,139.665626,5.0,6,"代田6-3-27, アゼリアハウス1F, Setagaya, 東京都 〒155-0033, ..."
